In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from keras.models import load_model

Read in the data

In [ ]:
# you can set this to True to test the Fixed Fourier Power data
FFP = False
ffpname = ''
if FFP:
    ffpname='_kp'

batch_size = 128
num_classes = 2

#load in the test data from both snapshots
path = '/Users/jegpeek/Dropbox/turb_sim_ism_DL/'
test_x_b1_1 = np.load(path+'b1p.01XY_test'+ffpname+'_ss1_sh_64_ims_128.npy')
test_x_b1_2 = np.load(path+'b1p.01XY_test'+ffpname+'_ss1_sh_64_ims_128_2.npy')
test_x_b1 = np.concatenate((test_x_b1_1, test_x_b1_2), axis=2)

test_x_b01_1 = np.load(path+'b.1p.01XY_test'+ffpname+'_ss1_sh_64_ims_128.npy')
test_x_b01_2 = np.load(path+'b.1p.01XY_test'+ffpname+'_ss1_sh_64_ims_128_2.npy')
test_x_b01 = np.concatenate((test_x_b01_1, test_x_b01_2), axis=2)

teshape = test_x_b1.shape

img_rows, img_cols = teshape[0], teshape[1]

x_test = np.zeros([teshape[2]*2, img_rows, img_rows])
x_test[::2, :, :] = np.transpose(test_x_b01)
x_test[1::2, :, :] = np.transpose(test_x_b1)

y_test = np.zeros([teshape[2]*2])
y_test[::2] = np.zeros([teshape[2]])
y_test[1::2] = np.ones([teshape[2]])


The data need a little more reshaping depending on how you have Keras set up

In [ ]:
if K.image_data_format() == 'channels_first':
    
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


x_test = x_test.astype('float32')

# convert class vectors to binary class matrices
y_test = keras.utils.to_categorical(y_test, num_classes)



Load the network

In [ ]:
modelname = 'PB19_network.dms'
if FFP:
    modelname = 'PB19_FFP_network.dms'

model = load_model(modelname)

Test the network

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])